In [2]:
"""
Created on Fir Feb  25 22:00:00 2022

@author: hanlong liao
"""
import json
import csv
import gzip
import pandas as pd
import numpy as np
import pickle
import re

* 第1步	提取目标变量（3.1，4，5，6）表格	
* 第2步	确定纳入patientunitstayid（表之间进行匹配的依据）	
* 第3步	多个来源的数据合并（生命体征、检验数据存储于多个表格）	
	同类项合并（药物数据）	
* 第4步	数据特征分析	
* 第5步	按时间段提取生命体征、检验数据（D1-D7,入ICU48h内）	
	...。。。。。	

# patient表格

In [90]:
# 处理patient表格
patient = pd.read_csv('./data/patient.csv', encoding='utf-8')

# patient 列字段名称
# Index(['patientunitstayid', 'patienthealthsystemstayid', 'gender', 'age',
#        'ethnicity', 'hospitalid', 'wardid', 'apacheadmissiondx',
#        'admissionheight', 'hospitaladmittime24', 'hospitaladmitoffset',
#        'hospitaladmitsource', 'hospitaldischargeyear',
#        'hospitaldischargetime24', 'hospitaldischargeoffset',
#        'hospitaldischargelocation', 'hospitaldischargestatus', 'unittype',
#        'unitadmittime24', 'unitadmitsource', 'unitvisitnumber', 'unitstaytype',
#        'admissionweight', 'dischargeweight', 'unitdischargetime24',
#        'unitdischargeoffset', 'unitdischargelocation', 'unitdischargestatus',
#        'uniquepid'],
#       dtype='object')
# patient.dtypes
patient = patient[['patientunitstayid', 'patienthealthsystemstayid', 'uniquepid', 'age', 'ethnicity', 'admissionweight', 'admissionheight', 'hospitaladmitoffset', 'hospitaladmitsource', 'hospitaldischargeyear', 'hospitaldischargeoffset', 'hospitaldischargestatus', 'unittype', 'unitdischargeoffset', 'unitdischargestatus']] #取所需字段
def age_rule(frame): 
    if (frame['age'] == '> 89') or (frame['age'] == '>89'):
        return '100'
    if pd.isnull(frame['age']) == True or pd.isna(frame['age'] == True):
        return '0'
    return frame['age']
patient['age'] = patient.apply(lambda x:age_rule(x), axis=1) # 年龄> 89的病人统一赋值100 和 空值赋值为0
# 筛选入院时间 > 24h 和年龄 > 14的病人
patient = patient[((patient['hospitaldischargeoffset'] - patient['hospitaladmitoffset']) / 1440 >= 1) \
    & (patient['age'].astype(dtype='int') > 14)]
# patient
patient.to_csv('./data/patient_NEW.csv', sep=',', index=False, header=True, encoding='utf_8_sig')

In [91]:
# 处理apachePatientResult表格
result_ = pd.read_csv('./data/apachePatientResult.csv', encoding='utf-8')
# result_ = result_[['acutephysiologyscore', 'apachescore', 'apacheversion', 'predictedicumortality', 'actualicumortality', 'unitdischargestatus', \
#     'predictediculos', 'actualiculos', 'unabridgedunitlos', 'predictedhospitalmortality', 'actualhospitalmortality', 'hospitaldischargestatus', \
#         'predictedhospitallos', 'unabridgedhosplos', 'actualhospitallos', 'unabridgedActualVentdays']]
# "['hospitaldischargestatus', 'unabridgedActualVentdays', 'unitdischargestatus'] not in index"
sorted(result_.columns)

['actualhospitallos',
 'actualhospitalmortality',
 'actualiculos',
 'actualicumortality',
 'actualventdays',
 'acutephysiologyscore',
 'apachepatientresultsid',
 'apachescore',
 'apacheversion',
 'patientunitstayid',
 'physicianinterventioncategory',
 'physicianspeciality',
 'predictedhospitallos',
 'predictedhospitalmortality',
 'predictediculos',
 'predictedicumortality',
 'predventdays',
 'preopcardiaccath',
 'preopmi',
 'ptcawithin24h',
 'unabridgedactualventdays',
 'unabridgedhosplos',
 'unabridgedunitlos']

In [92]:
# 处理NurseCharting表格
# nurseCharting表格过大，无法直接读取，因此循环读取，避免内存崩溃
nursechart = pd.read_csv('./data/nurseCharting.csv', encoding='utf-8', iterator=True, names=['nursingchartid', 'patientunitstayid', 'nursingchartoffset',
       'nursingchartentryoffset', 'nursingchartcelltypecat',
       'nursingchartcelltypevallabel', 'nursingchartcelltypevalname',
       'nursingchartvalue', 'filed1', 'filed2'])
loop = True
chunksize = 100000
chunks = []
index = 0
nursingcharttype = ['Respiratory Rate', 'Heart Rate', 'Temperature (F)', 'Temperature (C)', 'O2 Saturation', 'Non-Invasive BP Systolic', 'Non-Invasive BP Diastolic', \
'Invasive BP Systolic', 'Invasive BP Diastolic', 'Non-Invasive BP Mean', 'Invasive BP Mean']

print('Starting the Interation and Merging...')
while loop:
    try:
        chunk = nursechart.get_chunk(chunksize)
        chunk = chunk[['patientunitstayid', 'nursingchartoffset', 'nursingchartcelltypevalname', 'nursingchartvalue']]
        chunk_ = chunk[chunk['nursingchartcelltypevalname'].isin(nursingcharttype)]
        chunks.append(chunk_)
        index += 1
        print(index)
        if index % 15 == 0:
            data = pd.concat(chunks, ignore_index=True)
            outputpath = './data/nurseCharting_NEW/nurseCharting_NEW' + str(int(index / 15)) + '.csv'
            data.to_csv(outputpath, sep=',', index=False, header=True, encoding='utf-8_sig')
            chunks = []    
    except StopIteration:
        loop = False
        data = pd.concat(chunks, ignore_index=True)
        outputpath = './data/nurseCharting_NEW/nurseCharting_NEW' + str(int(index / 15) + 1) + '.csv'
        data.to_csv(outputpath, sep=',', index=False, header=True, encoding='utf-8_sig')
        print('The Interation and Merging Is Stopped!')

Starting the Interation and Merging...


C:\Users\dell\anaconda3\lib\site-packages\IPython\core\async_helpers.py:129: DtypeWarning: Columns (0,1,2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  coro.send(None)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


KeyboardInterrupt: 

In [ ]:
# 静脉用药剂量infusiondrug表格处理
infusiondrug = pd.read_csv('./data/infusionDrug.csv')
# infusiondrug 表格字段
# Index(['infusiondrugid', 'patientunitstayid', 'infusionoffset', 'drugname',
#        'drugrate', 'infusionrate', 'drugamount', 'volumeoffluid',
#        'patientweight'],
#       dtype='object')
infusiondrug = infusiondrug[['patientunitstayid', 'drugname', 'drugrate', 'infusionrate', 'drugamount', 'volumeoffluid']]

# 所要筛选出来的药物
# 血管活性类: 'epinephrine', 'isoproterenol', 'norepinephrine', 'phenylephrine', 'norepinephrine', 'nitroGLYCERIN', 'levophed', 'isoproterenol', 'dopamine';
# 正性肌力: 'amiodarone', 'brevibloc', 'vasopressin', 'milrinone', 'esmolol';
# 抗生素: 'zosyn', 'vancomycin', 'meropenem', 'cefazolin', 'ciprofloxacin', 'ceftriaxone';
# 麻醉: 'cisatracurium', 'argatroban', 'dexmedetomidine', 'zemuron', 'vecuronium', 'rocuronium', 'propofol', 'nimbex', 'midazolam';
# 营养: 'LIPIDS', 'fat emulsion', 'tpn';
# 激素: 'methylprednisolone', 'solumedrol';

drugname_name = ['epinephrine', 'isoproterenol', 'norepinephrine', 'phenylephrine', 'norepinephrine', 'nitroGLYCERIN', 'levophed', 'isoproterenol', 'dopamine', 'amiodarone', 'brevibloc', 'vasopressin', 'milrinone', 'esmolol', 'zosyn', 'vancomycin', 'meropenem', 'cefazolin', 'ciprofloxacin', 'ceftriaxone', 'cisatracurium', 'argatroban', 'dexmedetomidine', 'zemuron', 'vecuronium', 'rocuronium', 'propofol', 'nimbex', 'midazolam', 'LIPIDS', 'fat emulsion', 'tpn', 'methylprednisolone', 'solumedrol']
drugname_pattern = 'epinephrine|isoproterenol|norepinephrine|phenylephrine|norepinephrine|nitroGLYCERIN|levophed|isoproterenol|dopamine|amiodarone|brevibloc|vasopressin|milrinone|esmolol|zosyn|vancomycin|meropenem|cefazolin|ciprofloxacin|ceftriaxone|cisatracurium|argatroban|dexmedetomidine|zemuron|vecuronium|rocuronium|propofol|nimbex|midazolam|LIPIDS|fat emulsion|tpn|methylprednisolone|solumedrol'

# 匹配函数
def drug_rule(frame):
    # print(frame['drugname'])
    match_result = re.findall(drugname_pattern, str(frame['drugname']).lower())
    if match_result:
        return match_result[0]
    return frame['drugname']
infusiondrug['drugname_newname'] = infusiondrug.apply(lambda x:drug_rule(x), axis=1) # 匹配药物名称
infusiondrug = infusiondrug[infusiondrug['drugname_newname'].isin(drugname_name)] # 筛选所需药物

# infusiondrug.to_csv('./data/infusionDrug_NEW.csv', sep=',', index=False, header=True, encoding='utf_8_sig')

C:\Users\dell\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
drugname_unit = dict()
for k in drugname_name:
    drugname_unit[k] = list()
for index, row in infusiondrug.iterrows():
    unit = '()'
    drugname_ = row['drugname'].strip().split()
    if len(drugname_) > 1:
        unit = drugname_[-1]
    drugname_unit[row['drugname_newname']].append(unit)

In [ ]:
drugname_unit_ = dict() # 去除list中的重复单位
for key in drugname_unit.keys():
    drugname_unit_[key] = set(drugname_unit[key])

drugname_unit_count = dict() # 计数重复单位个数
for k in drugname_name: # 初始化
    drugname_unit_count[k] = list()
for key, value in drugname_unit_.items():
    for v in value:
        drugname_unit_count[key].append(v+str(drugname_unit[key].count(v)))

f = open('./data/drugname.txt', 'w+')
for key, value in drugname_unit_count.items():
    print(key)
    print(value)

epinephrine
['(mg/hr)9', '(mcg/hr)3', '(mcg/kg/min)3101', '(mcg/min)15239', '()2113', '(mg/kg/min)341', '(ml/hr)33272']
isoproterenol
['(mcg/min)1581', '()680', '(ml/hr)887']
norepinephrine
['(mg/hr)57', '(units/min)2', '(ml)95', '(mcg/kg/hr)58', '(mcg/hr)21', '(mcg/kg/min)68921', '(mg/min)8', '(mcg/min)197711', '()26447', '(mg/kg/min)9', '(Unknown)5', '(ml/hr)285793']
phenylephrine
['(mg/hr)3', '(mcg/hr)121', '(mcg/kg/min)16738', '(mcg/min)50697', '()13116', '(mg/kg/min)4', 'Phenylephrine44', '(ml/hr)60465']
nitroGLYCERIN
[]
levophed
['(mcg/kg/min)2', '(mg/hr)25', '(mcg/min)1512', '(ml/hr)190']
dopamine
['(mg/hr)2', '(mcg/kg/hr)4', '(mcg/hr)8', '(mcg/kg/min)29864', '()5403', '(mcg/min)3', '(nanograms/kg/min)1', '(Unknown)10', '(ml/hr)56077']
amiodarone
['(mg/hr)166', '(mg/min)71143', '(mcg/kg/min)1', '()19709', '(mcg/min)135', '(Unknown)8', '(ml/hr)127056']
brevibloc
['(mcg/kg/min)27']
vasopressin
['(units/hr)1006', '(units/min)62037', '(mg/hr)1', '(units/kg/min)1', '(mg/min)55', '(mc

In [ ]:
units = set()
for item in drugname_unit.values():
    for i in item:
        units.add(i)
units

{'()',
 '(Unknown)',
 '(mcg/hr)',
 '(mcg/kg/hr)',
 '(mcg/kg/min)',
 '(mcg/min)',
 '(mg/hr)',
 '(mg/kg/hr)',
 '(mg/kg/min)',
 '(mg/min)',
 '(ml)',
 '(ml/hr)',
 '(nanograms/kg/min)',
 '(units/hr)',
 '(units/kg/hr)',
 '(units/kg/min)',
 '(units/min)',
 'Argatroban',
 'Phenylephrine',
 'TPN',
 'Vecuronium'}

In [ ]:
# 静脉用药剂量infusiondrug表格处理
infusiondrug = pd.read_csv('./data/infusionDrug.csv')
# infusiondrug 表格字段
# Index(['infusiondrugid', 'patientunitstayid', 'infusionoffset', 'drugname',
#        'drugrate', 'infusionrate', 'drugamount', 'volumeoffluid',
#        'patientweight'],
#       dtype='object')
infusiondrug = infusiondrug[['patientunitstayid', 'drugname', 'drugrate', 'infusionrate', 'drugamount', 'volumeoffluid']]

# 所要筛选出来的药物
# 血管活性类: 'epinephrine', 'isoproterenol', 'norepinephrine', 'phenylephrine', 'norepinephrine', 'nitroGLYCERIN', 'levophed', 'isoproterenol', 'dopamine';
# 正性肌力: 'amiodarone', 'brevibloc', 'vasopressin', 'milrinone', 'esmolol';
# 抗生素: 'zosyn', 'vancomycin', 'meropenem', 'cefazolin', 'ciprofloxacin', 'ceftriaxone';
# 麻醉: 'cisatracurium', 'argatroban', 'dexmedetomidine', 'zemuron', 'vecuronium', 'rocuronium', 'propofol', 'nimbex', 'midazolam';
# 营养: 'LIPIDS', 'fat emulsion', 'tpn';
# 激素: 'methylprednisolone', 'solumedrol';

drugname_name = ['epinephrine', 'isoproterenol', 'norepinephrine', 'phenylephrine', 'norepinephrine', 'nitroGLYCERIN', 'levophed', 'isoproterenol', 'dopamine', 'amiodarone', 'brevibloc', 'vasopressin', 'milrinone', 'esmolol', 'zosyn', 'vancomycin', 'meropenem', 'cefazolin', 'ciprofloxacin', 'ceftriaxone', 'cisatracurium', 'argatroban', 'dexmedetomidine', 'zemuron', 'vecuronium', 'rocuronium', 'propofol', 'nimbex', 'midazolam', 'LIPIDS', 'fat emulsion', 'tpn', 'methylprednisolone', 'solumedrol']
drugname_pattern = 'epinephrine|isoproterenol|norepinephrine|phenylephrine|norepinephrine|nitroGLYCERIN|levophed|isoproterenol|dopamine|amiodarone|brevibloc|vasopressin|milrinone|esmolol|zosyn|vancomycin|meropenem|cefazolin|ciprofloxacin|ceftriaxone|cisatracurium|argatroban|dexmedetomidine|zemuron|vecuronium|rocuronium|propofol|nimbex|midazolam|LIPIDS|fat emulsion|tpn|methylprednisolone|solumedrol'
drugname_unit = dict()
for key in drugname_name:
    drugname_unit[key] = set()

# 匹配函数
def drug_rule(frame):
    match_result = re.findall(drugname_pattern, str(frame['drugname']).lower())
    if match_result:
        drugname_ = frame['drugname'].strip().split()
        unit = drugname_[-1]
        drugname_unit[match_result[0]].add(unit)
        return match_result[0]
    return frame['drugname']
infusiondrug['drugname_newname'] = infusiondrug.apply(lambda x:drug_rule(x), axis=1) # 匹配药物名称
infusiondrug = infusiondrug[infusiondrug['drugname_newname'].isin(drugname_name)] # 筛选所需药物

# infusiondrug.to_csv('./data/infusionDrug_NEW.csv', sep=',', index=False, header=True, encoding='utf_8_sig')
print(drugname_unit)

C:\Users\dell\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3098: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


{'epinephrine': {'(mcg/kg/min)', '(mcg/min)', '(ml/hr)', '(mg/hr)', 'Epinephrine', '(mcg/hr)', '(mg/kg/min)', '()'}, 'isoproterenol': {'Isoproterenol', '()', '(mcg/min)', '(ml/hr)'}, 'norepinephrine': {'(mcg/kg/min)', '(mg/min)', '(mcg/min)', '(ml/hr)', '(mg/hr)', '(units/min)', '(mcg/hr)', '(mg/kg/min)', '(Unknown)', '()', '(mcg/kg/hr)', '(ml)', 'Norepinephrine'}, 'phenylephrine': {'(mcg/kg/min)', '(mcg/min)', '(ml/hr)', '(mg/hr)', '(mcg/hr)', '(mg/kg/min)', '()', 'Phenylephrine'}, 'nitroGLYCERIN': set(), 'levophed': {'(mg/hr)', '(mcg/kg/min)', '(mcg/min)', '(ml/hr)'}, 'dopamine': {'(mcg/kg/min)', '(mcg/min)', '(ml/hr)', '(mg/hr)', '(nanograms/kg/min)', '(mcg/hr)', 'Dopamine', '(Unknown)', '()', '(mcg/kg/hr)'}, 'amiodarone': {'(mcg/kg/min)', '(mg/min)', '(mg/hr)', '(ml/hr)', 'amiodarone', '(mcg/min)', 'Amiodarone', '(Unknown)', '()'}, 'brevibloc': {'(mcg/kg/min)'}, 'vasopressin': {'(mcg/kg/min)', '(mg/min)', '(units/kg/min)', '(mcg/min)', '(ml/hr)', '(mg/hr)', '(Unknown)', '(units/min

In [149]:
# 静脉用药剂量infusiondrug表格处理
infusiondrug = pd.read_csv('./data/infusionDrug.csv')
# infusiondrug 表格字段
# Index(['infusiondrugid', 'patientunitstayid', 'infusionoffset', 'drugname',
#        'drugrate', 'infusionrate', 'drugamount', 'volumeoffluid',
#        'patientweight'],
#       dtype='object')
infusiondrug = infusiondrug[['patientunitstayid', 'drugname', 'drugrate', 'infusionrate', 'drugamount', 'volumeoffluid', 'patientweight', 'infusionoffset']]

# 所要筛选出来的药物
# 血管活性类: 'epinephrine', 'isoproterenol', 'norepinephrine', 'phenylephrine', 'norepinephrine', 'nitroGLYCERIN', 'levophed', 'isoproterenol', 'dopamine';
# 正性肌力: 'amiodarone', 'brevibloc', 'vasopressin', 'milrinone', 'esmolol';
# 抗生素: 'zosyn', 'vancomycin', 'meropenem', 'cefazolin', 'ciprofloxacin', 'ceftriaxone';
# 麻醉: 'cisatracurium', 'argatroban', 'dexmedetomidine', 'zemuron', 'vecuronium', 'rocuronium', 'propofol', 'nimbex', 'midazolam';
# 营养: 'LIPIDS', 'fat emulsion', 'tpn';
# 激素: 'methylprednisolone', 'solumedrol';

drugname_name = ['epinephrine', 'isoproterenol', 'norepinephrine', 'phenylephrine', 'norepinephrine', 'nitroGLYCERIN', 'levophed', 'isoproterenol', 'dopamine', 'amiodarone', 'brevibloc', 'vasopressin', 'milrinone', 'esmolol', 'zosyn', 'vancomycin', 'meropenem', 'cefazolin', 'ciprofloxacin', 'ceftriaxone', 'cisatracurium', 'argatroban', 'dexmedetomidine', 'zemuron', 'vecuronium', 'rocuronium', 'propofol', 'nimbex', 'midazolam', 'LIPIDS', 'fat emulsion', 'tpn', 'methylprednisolone', 'solumedrol']
drugname_pattern = 'epinephrine|isoproterenol|norepinephrine|phenylephrine|norepinephrine|nitroGLYCERIN|levophed|isoproterenol|dopamine|amiodarone|brevibloc|vasopressin|milrinone|esmolol|zosyn|vancomycin|meropenem|cefazolin|ciprofloxacin|ceftriaxone|cisatracurium|argatroban|dexmedetomidine|zemuron|vecuronium|rocuronium|propofol|nimbex|midazolam|LIPIDS|fat emulsion|tpn|methylprednisolone|solumedrol'

# 匹配函数,筛选需要的药物种类
def drug_rule(frame):
    match_result = re.findall(drugname_pattern, str(frame['drugname']).lower())
    if match_result:
        return match_result[0]
    return None
infusiondrug['drugname_newname'] = infusiondrug.apply(lambda x:drug_rule(x), axis=1) # 匹配药物名称
infusiondrug = infusiondrug[infusiondrug['drugname_newname'].isin(drugname_name)] # 筛选所需药物

# 提取单位
def unit_rule(frame):
    drugname_ = frame['drugname'].strip().split()
    if drugname_:
        return drugname_[-1]
    else:
        return None

infusiondrug['unit'] = infusiondrug.apply(lambda x:unit_rule(x), axis=1)
infusiondrug.to_csv('infusiondrug_new.csv', sep=',', index=False, header=True, encoding='utf_8_sig')

C:\Users\dell\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3098: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


KeyboardInterrupt: 

In [71]:
# 初步清洗数据
infusiondrug = pd.read_csv('infusiondrug_new.csv')
# infusiondrug
# drugrate 中存在 '>=' 或者 '<=' 字符,需要提取出其中的数字
def drugrate_rule(frame):
    rate = re.findall(r'[0-9]+\.?[0-9]*', str(frame['drugrate']))
    if rate:
        return rate[0]
    else:
        return '0'
    
infusiondrug['drugrate_new'] = infusiondrug.apply(lambda x:drugrate_rule(x), axis=1) 
infusiondrug['drugrate_new'] = pd.to_numeric(infusiondrug['drugrate_new'], errors='coerce').fillna(0) # 利用中间函数to_numeric来强制转化数据类型，防止因为空值以及其它问题出现的类型转化失败
infusiondrug = infusiondrug[(infusiondrug['drugrate_new'].notnull()) & (infusiondrug['drugrate_new'] != 0)]

C:\Users\dell\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3098: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [72]:
# 统一单位
# 运行大概10分钟
units = ['(mcg/min)', '(ml/hr)', '(units/min)', '(mcg/kg/min)', '(mcg/hr)', '(mcg/kg/hr)', '(nanograms/kg/min)', '(mg/hr)', '(mg/min)', '(mg/kg/hr)', '(mg/kg/min)', '(units/hr)', '(units/kg/hr)']

def num_unifine(frame):
    r = frame['drugrate_new']
    unit = str(frame['unit']).lower()

    if unit not in units:
        unit = '(ml/hr)'
    w = frame['patientweight']

    # 以下单位无需换算
    if unit in (['(mcg/min)', '(ml/hr)', '(units/min)']):
        return pd.Series([r, unit])

    # 以下单位换算成mcg/min
    elif unit == '(mcg/kg/min)':
        return pd.Series([r * w, '(mcg/min)'])
    elif unit == '(mcg/hr)':
        return pd.Series([r / 60, '(mcg/min)'])
    elif unit == '(mcg/kg/hr)':
        return pd.Series([r * w / 60, '(mcg/min)'])
    elif unit == '(nanograms/kg/min)':
        return pd.Series([r * w / 1000, '(mcg/min)'])
    elif unit == '(mg/hr)':
        return pd.Series([r * 17, '(mcg/min)'])
    elif unit == '(mg/min)':
        return pd.Series([r * 1000, '(mcg/min)'])
    elif unit == '(mg/kg/hr)':
        return pd.Series([r * 1000 * w / 60, '(mcg/min)'])
    elif unit == '(mg/kg/min)':
        return pd.Series([r * 1000, '(mcg/min)'])
    
    # 以下单位换算成units/min
    elif unit == '(units/hr)':
        return pd.Series([r / 60, '(units/min)'])
    elif unit == '(units/kg/hr)':
        return pd.Series([r * w / 60, '(units/min)'])
    else:
        return pd.Series([None, None])

infusiondrug[['drugrate_trans', 'unit_trans']] = infusiondrug.apply(lambda x:num_unifine(x), axis=1)
infusiondrug.to_csv('./infusiondrug_new_trans.csv', sep=',', index=False, header=True, encoding='utf-8') # 所有单位统一

In [150]:
infusiondrug = pd.read_csv('./infusiondrug_new_trans.csv')
patient = pd.read_csv('./data/patient_NEW.csv')
infusiondrug_new = pd.merge(infusiondrug, patient, on=['patientunitstayid'], how='inner')
def daytime(frame):
    day = int(((frame['hospitaldischargeoffset'] - frame['infusionoffset']) / 1440)) + 1
    return day

infusiondrug_new['daytime'] = infusiondrug_new.apply(lambda x:daytime(x), axis=1) 
infusiondrug_new = infusiondrug_new[(infusiondrug_new['daytime'] <= 7) & (infusiondrug_new['daytime'] >= 1)]

infusiondrug_new = infusiondrug_new[['patientunitstayid', 'drugname_newname', 'unit_trans', 'drugrate_trans', 'daytime']]
infusiondrug_new = infusiondrug_new.pivot_table('drugrate_trans', index=['patientunitstayid'], columns=['drugname_newname', 'unit_trans', 'daytime'], aggfunc=[np.max, ])

C:\Users\dell\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3098: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [151]:
# 添加缺失列
infusiondrug_new_ = infusiondrug_new.copy(deep=True)
infusiondrug_new_.columns = infusiondrug_new.columns.droplevel(0)

lev1, lev2, lev3 = zip(*infusiondrug_new_.columns.tolist())
index_dict = dict()
index_zip = zip(zip(lev1, lev2), lev3)
for key, value in index_zip:
    if key not in index_dict.keys():
        index_dict[key] = list()
    index_dict[key].append(value)

index_dict

for key, value in index_dict.items():
    if set(value) != set([1, 2, 3, 4, 5, 6, 7]):
        print(key, value)
        value_ = list(set([1, 2, 3, 4, 5, 6, 7]) - set(value))
        print(value_)
        for v in value_:
            infusiondrug_new_.loc[:, (key[0], key[1], v)] = np.nan

infusiondrug_new_ = infusiondrug_new_.sort_index(axis=1)
infusiondrug_new_ 

('brevibloc', '(mcg/min)') [4, 5]
[1, 2, 3, 6, 7]
('ceftriaxone', '(ml/hr)') [4, 7]
[1, 2, 3, 5, 6]
('fat emulsion', '(ml/hr)') [5]
[1, 2, 3, 4, 6, 7]
('levophed', '(ml/hr)') [2, 3, 4, 5, 6, 7]
[1]
('meropenem', '(ml/hr)') [3, 4, 5, 6]
[1, 2, 7]
('methylprednisolone', '(ml/hr)') [6]
[1, 2, 3, 4, 5, 7]
('nimbex', '(ml/hr)') [1, 4]
[2, 3, 5, 6, 7]
('solumedrol', '(mcg/min)') [3, 4]
[1, 2, 5, 6, 7]
('tpn', '(mcg/min)') [5, 6]
[1, 2, 3, 4, 7]
('vancomycin', '(ml/hr)') [2, 3, 4, 5, 6, 7]
[1]
('vasopressin', '(mcg/min)') [1, 2, 5, 7]
[3, 4, 6]
('zemuron', '(mcg/min)') [1, 7]
[2, 3, 4, 5, 6]


drugname_newname  amiodarone                                          ...  \
unit_trans         (mcg/min)                         (ml/hr)          ...   
daytime                    1   2   3   4   5   6   7       1   2   3  ...   
patientunitstayid                                                     ...   
242040                   NaN NaN NaN NaN NaN NaN NaN     NaN NaN NaN  ...   
242082                   NaN NaN NaN NaN NaN NaN NaN     NaN NaN NaN  ...   
242154                   NaN NaN NaN NaN NaN NaN NaN     NaN NaN NaN  ...   
242319                   NaN NaN NaN NaN NaN NaN NaN     NaN NaN NaN  ...   
242505                   NaN NaN NaN NaN NaN NaN NaN     NaN NaN NaN  ...   
...                      ...  ..  ..  ..  ..  ..  ..     ...  ..  ..  ...   
3352895                  NaN NaN NaN NaN NaN NaN NaN     NaN NaN NaN  ...   
3352970                  NaN NaN NaN NaN NaN NaN NaN     NaN NaN NaN  ...   
3353097                  NaN NaN NaN NaN NaN NaN NaN     NaN NaN NaN  ...   
3353117                  NaN NaN NaN NaN NaN NaN NaN     NaN NaN NaN  ...   
3353123                  NaN NaN NaN NaN NaN NaN NaN     NaN NaN NaN  ...   

drugname_newname    zemuron           zosyn                          
unit_trans        (mcg/min)         (ml/hr)                          
daytime                   5   6   7       1   2   3   4   5   6   7  
patientunitstayid                                                    
242040                  NaN NaN NaN     NaN NaN NaN NaN NaN NaN NaN  
242082                  NaN NaN NaN     NaN NaN NaN NaN NaN NaN NaN  
242154                  NaN NaN NaN     NaN NaN NaN NaN NaN NaN NaN  
242319                  NaN NaN NaN     NaN NaN NaN NaN NaN NaN NaN  
242505                  NaN NaN NaN     NaN NaN NaN NaN NaN NaN NaN  
...                     ...  ..  ..     ...  ..  ..  ..  ..  ..  ..  
3352895                 NaN NaN NaN     NaN NaN NaN NaN NaN NaN NaN  
3352970                 NaN NaN NaN     NaN NaN NaN NaN NaN NaN NaN  
3353097                 NaN NaN NaN     NaN NaN NaN NaN NaN NaN NaN  
3353117                 NaN NaN NaN     NaN NaN NaN NaN NaN NaN NaN  
3353123                 NaN NaN NaN     NaN NaN NaN NaN NaN NaN NaN  

[26651 rows x 336 columns]

In [ ]:
# 处理成最终的数据集
infusiondrug = pd.read_csv('./infusiondrug_new_trans.csv')
patient = pd.read_csv('./data/patient_NEW.csv')
infusiondrug_new = pd.merge(infusiondrug, patient, on=['patientunitstayid'], how='inner')
infusiondrug_new = infusiondrug_new[['patientunitstayid', 'drugname_newname', 'unit_trans', 'drugrate_trans']]

infusiondrug_new_ = infusiondrug_new.groupby(['patientunitstayid', 'drugname_newname', 'unit_trans']).sum().to_dict()['drugrate_trans']
infusiondrug_final = infusiondrug_new['patientunitstayid'].drop_duplicates().to_frame().set_index('patientunitstayid')
infusiondrug_final['drug_rate'] = ''
# infusiondrug_final

for key, value in infusiondrug_new_.items():
    patientid = key[0]
    drugrate = value
    drug_cate = key[1] + ' ' + key[2] + ' '+ str(value)
    ori_rate = infusiondrug_final.loc[patientid, 'drug_rate']
    infusiondrug_final.loc[patientid, 'drug_rate'] = ori_rate + ', ' + drug_cate
infusiondrug_final

C:\Users\dell\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3098: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,drug_rate
patientunitstayid,
242040,", milrinone (mcg/min) 3943.1"
242082,", norepinephrine (mcg/min) 50.129999999999995"
242154,", propofol (mcg/min) 796.5"
242319,", norepinephrine (mcg/min) 4.0"
242505,", norepinephrine (mcg/min) 99.0"
...,...
3353199,", norepinephrine (mcg/min) 110.0, propofol (mc..."
3353200,", dopamine (mcg/min) 0.0"
3353216,", propofol (mcg/min) 0.0"


In [28]:
drugname_statics = []
for key_name in drugname_name:
    # print('########################################################### ' + key_name + ' #######################################################################')
    df = infusiondrug[infusiondrug['drugname_newname'] == key_name]
    df_unique = df['drugname'].unique().tolist() # 同一类药的不重复列表
    for d in df_unique:
        # print("----------------------"+ d +"---------------------------")
        df_unique_ = df[df['drugname'] == d]
        list_view = df_unique_['drugrate'].describe()
        # print('Count: {}, Max: {}, Min: {}, Mean: {}, Median: {}, IQR: {}'.format(list_view['count'], list_view['max'],  list_view['min'], list_view['mean'], list_view['50%'], list_view['75%'] - list_view['25%']))
        drugname_statics.append([key_name, d, list_view['count'], list_view['max'],  list_view['min'], list_view['mean'], list_view['50%'], list_view['75%'] - list_view['25%']])

pd.DataFrame(drugname_statics, columns=['Drugcategory', 'Drugname', 'Count', 'Max', 'Min', 'Mean', 'Median', 'IQR']).to_csv('./drug_statics.csv')

# Intakeoutput 

In [41]:
# 出入量（尿液）intakeOutput表格处理
intakeoutput = pd.read_csv('./data/intakeOutput.csv')
# Index(['intakeoutputid', 'patientunitstayid', 'intakeoutputoffset',
#        'intaketotal', 'outputtotal', 'dialysistotal', 'nettotal',
#        'intakeoutputentryoffset', 'cellpath', 'celllabel', 'cellvaluenumeric',
#        'cellvaluetext'],
#       dtype='object')
intakeoutput = intakeoutput[['patientunitstayid', 'intakeoutputoffset', 'intaketotal', 'outputtotal', 'dialysistotal', 'nettotal', 'celllabel', 'cellvaluenumeric']]
intakeoutput = intakeoutput[(pd.isnull(intakeoutput['celllabel']) == False) & (intakeoutput['intaketotal'])] # celllable不为空

patient_new = pd.read_csv('./data/patient_NEW.csv')
# Index(['patientunitstayid', 'patienthealthsystemstayid', 'uniquepid', 'age',
#        'ethnicity', 'admissionweight', 'admissionheight',
#        'hospitaladmitoffset', 'hospitaladmitsource', 'hospitaldischargeyear',
#        'hospitaldischargeoffset', 'hospitaldischargestatus', 'unittype',
#        'unitdischargeoffset', 'unitdischargestatus'],
#       dtype='object')

In [42]:
output = pd.merge(intakeoutput, patient_new, on=['patientunitstayid'], how='inner')

def daytime(frame):
    day = int(((frame['hospitaldischargeoffset'] - frame['intakeoutputoffset']) / 1440)) + 1
    return day

output['daytime'] = output.apply(lambda x:daytime(x), axis=1)

In [44]:
output.to_csv('./intakeoutput_new.csv', sep=',', index=False, header=True, encoding='utf_8')

In [64]:
intakeoutput_new = pd.read_csv('./intakeoutput_new.csv')
intakeoutput_new
intakeoutput_new = intakeoutput_new[['patientunitstayid', 'daytime', 'outputtotal']]
output_sum = intakeoutput_new.groupby(['patientunitstayid', 'daytime']).sum().to_dict()['outputtotal']

output_final = intakeoutput_new['patientunitstayid'].drop_duplicates().to_frame().set_index('patientunitstayid')
output_final[['1', '2', '3', '4', '5', '6', '7']] = ''
for key, value in output_sum.items():
    id = key[0]
    day = key[1]
    if day <= 7 and day >= 1:
        output_final.loc[id, str(day)] = value 

In [65]:
output_final

,1,2,3,4,5,6,7
patientunitstayid,,,,,,,
150708,1400.0,450.0,1650.0,0.0,2455.0,1880.0,4100.0
158159,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200733,0.0,3000.0,5650.0,0.0,8000.0,6900.0,3800.0
235783,500.0,1000.0,4025.0,0.0,1600.0,600.0,0.0
178962,500.0,0.0,200.0,1520.0,840.0,1100.0,0.0
...,...,...,...,...,...,...,...
3333263,20.0,0.0,,,,,
3332812,0.0,,,,,,
3347574,,120.0,,,,,


In [66]:
output_final.to_csv('./intakeoutput_final.csv')

In [39]:
output_ = output[['patientunitstayid', 'daytime', 'outputtotal']]
outputgroup_max = output_.groupby(['patientunitstayid', 'daytime']).sum()
# outputgroup_min = output_.groupby(['patientunitstayid', 'daytime']).agg('min')
outputgroup = pd.merge(outputgroup_min, outputgroup_max, on=['patientunitstayid', 'daytime'], how='inner')
outputgroup

NameError: name 'output' is not defined

In [ ]:
output_ = output[['patientunitstayid', 'intakeoutputoffset', 'daytime']]
outputgroup_max = output_.groupby(['patientunitstayid', 'daytime']).agg('max')
outputgroup_min = output_.groupby(['patientunitstayid', 'daytime']).agg('min')
outputgroup = pd.merge(outputgroup_min, outputgroup_max, on=['patientunitstayid', 'daytime'], how='inner')

def get_mean_io(frame):
    

#  lab表格

In [11]:
# lab数据提取与初筛
# TODO: 是否需要统一单位
lab = pd.read_csv('./data/lab.csv')
patient = pd.read_csv('./data/patient_NEW.csv')
patient = patient[['patientunitstayid', 'hospitaldischargeoffset']]
# customlab = pd.read_csv('./data/customLab.csv')
lab = lab[['patientunitstayid', 'labresultoffset', 'labname', 'labresult', 'labmeasurenamesystem']]
labname_pattern1 = ['WBC x 1000', 'RBC', 'platelets x 1000', 'RDW', 'MCV', 'MPV', 'Hgb', 'Hct', 'MCH', 'MCHC', 'CRP', \
    'CRP-hs', 'ESR', 'ALT (SGPT)', 'AST (SGOT)', 'total bilirubin', 'direct bilirubin', 'total protein', 'albumin', \
        'Globulin', 'BUN', 'creatinine', 'uric acid', 'CPK', 'CPK-MB', 'LDH', 'CPK-MB INDEX', 'myoglobin', 'potassium', \
            'sodium', 'chloride', 'calcium', 'anion gap', 'PT', 'PTT', 'PT - INR', 'fibrinogen', 'PTT ratio', 'FiO2', \
                'pH', 'paCO2', 'paO2', 'FiO2', 'HCO3', 'bicarbonate', 'Base Excess', 'O2 Sat (%)', 'lactate', 'glucose', 'bedside glucose'\
                    'total cholesterol', 'LDL', 'HDL']
lab = pd.merge(patient, lab, on=['patientunitstayid'], how='inner')
lab = lab[lab['labname'].isin(labname_pattern1)]
# lab = pd.merge(lab, customlab, on=['patientunitstayid'], how='inner')

# 包含"ab neu"
# 包含"neu",但不包含ab
# def labname_rule(frame):
#     if frame['labname'] in labname_pattern1:
#         return True
#     if re.search('neu', frame['labname']):
#         return True
#     return False
    
# lab['labname_rule'] = lab.apply(lambda x:labname_rule(x), axis=1)
# lab = lab[lab['labname_rule'] == True]

def daytime(frame):
    day = int(((frame['hospitaldischargeoffset'] - frame['labresultoffset']) / 1440)) + 1
    return day

lab['daytime'] = lab.apply(lambda x:daytime(x), axis=1)
lab.to_csv('./data/lab_NEW.csv', sep=',', index=False, header=True, encoding='utf_8_sig')

In [12]:
# 对表格数据类型进行初筛
lab_new = pd.read_csv('./data/lab_NEW.csv')
lab_new = lab_new[lab_new['labresult'].notnull()]
def labresult_rule(frame):
    rate = re.findall(r'[0-9]+\.?[0-9]*', str(frame['labresult']))
    if rate:
        return rate[0]
    else:
        return '0'

lab_new['labresult'] = lab_new.apply(lambda x:labresult_rule(x), axis=1) # 提取数值
lab_new['labresult'] = pd.to_numeric(lab_new['labresult'], errors='coerce').fillna(0) # 强制类型转换
lab_new = lab_new[lab_new['labresult'] != 0] # 去除0值
lab_new.to_csv('./data/lab_num.csv', sep=',', index=False, header=True, encoding='utf_8_sig')

In [13]:
lab_new

,patientunitstayid,hospitaldischargeoffset,labresultoffset,labname,labresult,labmeasurenamesystem,daytime
0,141168,3596,2026,fibrinogen,177.00,mg/dL,2
1,141168,3596,1133,PT - INR,2.50,ratio,2
2,141168,3596,1133,PT,26.60,sec,2
3,141168,3596,2141,pH,7.20,NaN,2
4,141168,3596,231,PT - INR,1.70,ratio,3
...,...,...,...,...,...,...,...
28833255,3353263,9590,1733,Hgb,12.90,g/dL,6
28833256,3353263,9590,-7,WBC x 1000,6.40,K/mcL,7
28833257,3353263,9590,1733,RBC,4.67,M/mcL,6
28833258,3353263,9590,1733,WBC x 1000,6.60,K/mcL,6


In [14]:
lab_new.dtypes

patientunitstayid            int64
hospitaldischargeoffset      int64
labresultoffset              int64
labname                     object
labresult                  float64
labmeasurenamesystem        object
daytime                      int64
dtype: object

In [26]:
# 对数据进行分类存储
lab_new = pd.read_csv('./data/lab_num.csv')
lab_new = lab_new[['patientunitstayid', 'labname', 'labresult', 'labmeasurenamesystem', 'daytime']]
lab_new_group = lab_new.groupby(['patientunitstayid', 'labname', 'daytime'])['labresult'].agg([np.mean, np.max, np.min])

In [35]:
lab_new_index = lab_new_group.reset_index()

In [49]:
# 按照每一天来处理表格
labname_pattern2 = ['WBC x 1000', 'RBC', 'platelets x 1000', 'RDW', 'MCV', 'MPV', 'Hgb', 'Hct', 'MCH', 'MCHC', 'CRP', \
    'CRP-hs', 'ESR', 'ALT (SGPT)', 'AST (SGOT)', 'total bilirubin', 'direct bilirubin', 'total protein', 'albumin', \
        'Globulin', 'BUN', 'creatinine', 'uric acid', 'CPK', 'CPK-MB', 'LDH', 'CPK-MB INDEX', 'myoglobin', 'potassium', \
            'sodium', 'chloride', 'calcium', 'anion gap', 'PT', 'PTT', 'PT - INR', 'fibrinogen', 'PTT ratio', 'FiO2', \
                'pH', 'paCO2', 'paO2', 'FiO2', 'HCO3', 'bicarbonate', 'Base Excess', 'O2 Sat (%)', 'lactate', 'glucose', 'bedside glucose'\
                    'total cholesterol', 'LDL', 'HDL']

labset = []
for item in labname_pattern2:
    lab_ = lab_new_index[lab_new_index['labname'] == item].copy(deep=True)

    lab_1 = lab_[lab_['daytime'] == 1].copy(deep=True)[['patientunitstayid', 'mean', 'amax', 'amin']].rename(columns={'mean': item + '_mean_1', 'amax': item + '_amax_1', 'amin': item + '_amin_1'}).set_index('patientunitstayid')
    lab_2 = lab_[lab_['daytime'] == 2].copy(deep=True)[['patientunitstayid', 'mean', 'amax', 'amin']].rename(columns={'mean': item + '_mean_2', 'amax': item + '_amax_2', 'amin': item + '_amin_2'}).set_index('patientunitstayid')
    lab_3 = lab_[lab_['daytime'] == 3].copy(deep=True)[['patientunitstayid', 'mean', 'amax', 'amin']].rename(columns={'mean': item + '_mean_3', 'amax': item + '_amax_3', 'amin': item + '_amin_3'}).set_index('patientunitstayid')
    lab_4 = lab_[lab_['daytime'] == 4].copy(deep=True)[['patientunitstayid', 'mean', 'amax', 'amin']].rename(columns={'mean': item + '_mean_4', 'amax': item + '_amax_4', 'amin': item + '_amin_4'}).set_index('patientunitstayid')
    lab_5 = lab_[lab_['daytime'] == 5].copy(deep=True)[['patientunitstayid', 'mean', 'amax', 'amin']].rename(columns={'mean': item + '_mean_5', 'amax': item + '_amax_5', 'amin': item + '_amin_5'}).set_index('patientunitstayid')
    lab_6 = lab_[lab_['daytime'] == 6].copy(deep=True)[['patientunitstayid', 'mean', 'amax', 'amin']].rename(columns={'mean': item + '_mean_6', 'amax': item + '_amax_6', 'amin': item + '_amin_6'}).set_index('patientunitstayid')
    lab_7 = lab_[lab_['daytime'] == 7].copy(deep=True)[['patientunitstayid', 'mean', 'amax', 'amin']].rename(columns={'mean': item + '_mean_7', 'amax': item + '_amax_7', 'amin': item + '_amin_7'}).set_index('patientunitstayid')

    labs = [lab_1, lab_2, lab_3, lab_4, lab_5, lab_6, lab_7]
    save_path = './data/LAB/' + item + '.csv'
    lab_item = pd.concat(labs, axis=1, sort=False, join='outer').reset_index().copy(deep=True)
    lab_item.to_csv(save_path, sep=',', header=True, index=False, encoding='utf_8_sig')
    labset.append(lab_item.set_index('patientunitstayid'))
    print('Finish labname {}...'.format(item))

# TODO:cusmlab表格有三项数据未添加

Finish labname WBC x 1000...
Finish labname RBC...
Finish labname platelets x 1000...
Finish labname RDW...
Finish labname MCV...
Finish labname MPV...
Finish labname Hgb...
Finish labname Hct...
Finish labname MCH...
Finish labname MCHC...
Finish labname CRP...
Finish labname CRP-hs...
Finish labname ESR...
Finish labname ALT (SGPT)...
Finish labname AST (SGOT)...
Finish labname total bilirubin...
Finish labname direct bilirubin...
Finish labname total protein...
Finish labname albumin...
Finish labname Globulin...
Finish labname BUN...
Finish labname creatinine...
Finish labname uric acid...
Finish labname CPK...
Finish labname CPK-MB...
Finish labname LDH...
Finish labname CPK-MB INDEX...
Finish labname myoglobin...
Finish labname potassium...
Finish labname sodium...
Finish labname chloride...
Finish labname calcium...
Finish labname anion gap...
Finish labname PT...
Finish labname PTT...
Finish labname PT - INR...
Finish labname fibrinogen...
Finish labname PTT ratio...
Finish lab

In [51]:
pd.concat(labset, axis=1, sort=False, join='outer').to_csv('./data/lab_final.csv', sep=',', index=False, header=True, encoding='utf_8_sig')

In [54]:
pd.concat(labset, axis=1, sort=False, join='outer')

,WBC x 1000_mean_1,WBC x 1000_amax_1,WBC x 1000_amin_1,WBC x 1000_mean_2,WBC x 1000_amax_2,WBC x 1000_amin_2,WBC x 1000_mean_3,WBC x 1000_amax_3,WBC x 1000_amin_3,WBC x 1000_mean_4,...,HDL_amin_4,HDL_mean_5,HDL_amax_5,HDL_amin_5,HDL_mean_6,HDL_amax_6,HDL_amin_6,HDL_mean_7,HDL_amax_7,HDL_amin_7
patientunitstayid,,,,,,,,,,,,,,,,,,,,,
141168,NaN,NaN,NaN,17.25,19.8,14.7,9.8,9.8,9.8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141178,NaN,NaN,NaN,7.60,7.6,7.6,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141179,8.1,8.1,8.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141194,8.9,8.9,8.9,8.70,8.7,8.7,6.9,6.9,6.9,4.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141196,12.7,12.7,12.7,14.20,14.2,14.2,NaN,NaN,NaN,26.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3353235,NaN,NaN,NaN,NaN,NaN,NaN,5.4,5.4,5.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3353237,NaN,NaN,NaN,5.70,5.7,5.7,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3353251,18.3,18.3,18.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Respiratory表格，机械通气

In [57]:
# 处理机械通气数据 respiratorycare表格处理
respiratory = pd.read_csv('./data/respiratoryCare.csv')
# Index(['respcareid', 'patientunitstayid', 'respcarestatusoffset',
#        'currenthistoryseqnum', 'airwaytype', 'airwaysize', 'airwayposition',
#        'cuffpressure', 'ventstartoffset', 'ventendoffset',
#        'priorventstartoffset', 'priorventendoffset', 'apneaparms',
#        'lowexhmvlimit', 'hiexhmvlimit', 'lowexhtvlimit', 'hipeakpreslimit',
#        'lowpeakpreslimit', 'hirespratelimit', 'lowrespratelimit',
#        'sighpreslimit', 'lowironoxlimit', 'highironoxlimit',
#        'meanairwaypreslimit', 'peeplimit', 'cpaplimit', 'setapneainterval',
#        'setapneatv', 'setapneaippeephigh', 'setapnearr', 'setapneapeakflow',
#        'setapneainsptime', 'setapneaie', 'setapneafio2'],
#       dtype='object')
respiratory = respiratory[['patientunitstayid', 'ventstartoffset', 'ventendoffset']]
respiratory.dtypes

C:\Users\dell\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3098: DtypeWarning: Columns (4,5,6,12,26,27,28,30,31,32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


patientunitstayid    int64
ventstartoffset      int64
ventendoffset        int64
dtype: object

In [59]:
# 基本数据清洗与加入时间戳计算最后7天
patient = pd.read_csv('./data/patient_NEW.csv')
respiratory_new = pd.merge(respiratory, patient, on=['patientunitstayid'], how='inner')

#TODO: 存在ventendoffset 小于 ventndoffset, 貌似机械通气在进入ICU之后就会关闭
respiratory_new = respiratory_new[respiratory_new['ventstartoffset'] < respiratory_new['ventendoffset']]

def daytime_rule(frame):
    day_end = int(((frame['hospitaldischargeoffset'] - frame['ventendoffset']) / 1440)) + 1
    day_start = int(((frame['hospitaldischargeoffset'] - frame['ventstartoffset']) / 1440)) + 1
    day_list= [0 for i in range(7)]
    if day_start <= 7 and day_end >= 1:
        for i in range(day_end-1, day_start-1):
            day_list[i] = 1
    return pd.Series(day_list)

respiratory_new[['res_day_1', 'res_day_2', 'res_day_3', 'res_day_4', 'res_day_5', 'res_day_6', 'res_day_7']] = respiratory_new.apply(lambda x:daytime_rule(x), axis=1)
respiratory_new = respiratory_new[['patientunitstayid', 'res_day_1', 'res_day_2', 'res_day_3', 'res_day_4', 'res_day_5', 'res_day_6', 'res_day_7']]
respiratory_new


,patientunitstayid,res_day_1,res_day_2,res_day_3,res_day_4,res_day_5,res_day_6,res_day_7
3,141233,0,0,0,0,0,0,0
5,141276,0,0,0,0,0,0,0
6,141289,0,0,0,0,0,0,0
7,141436,0,0,0,0,0,0,0
10,141451,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...
857915,3353251,0,0,0,0,0,0,0
857949,3353251,0,0,0,0,0,0,0
857958,3353251,0,0,0,0,0,0,0
857960,3353251,0,0,0,0,0,0,0


In [65]:
# 基本数据清洗与入ICU72小时通气情况
col_name = []
for i in range(1, 73):
    col_name.append('res_hours_' + str(i))
col_name_ = ['patientunitstayid', ] + col_name
patient = pd.read_csv('./data/patient_NEW.csv')
respiratory_hours = pd.merge(respiratory, patient, on=['patientunitstayid'], how='inner')

#TODO: 存在ventendoffset 小于 ventndoffset, 貌似机械通气在进入ICU之后就会关闭
respiratory_hours = respiratory_hours[respiratory_hours['ventstartoffset'] < respiratory_hours['ventendoffset']]

def hours_rule(frame):
    hours_list= [0 for i in range(72)]
    
    hours_start = int(frame['ventstartoffset'] / 60) + 1
    hours_end = int(frame['ventendoffset'] / 60) + 1
    if hours_start >= 0  and hours_end >= 0:    
        for i in range(hours_start - 1, hours_end - 1):
            hours_list[i] = 1

    return pd.Series(hours_list)

respiratory_hours[col_name] = respiratory_hours.apply(lambda x:hours_rule(x), axis=1)
respiratory_hours = respiratory_hours[col_name_]
respiratory_hours

,patientunitstayid,res_hours_1,res_hours_2,res_hours_3,res_hours_4,res_hours_5,res_hours_6,res_hours_7,res_hours_8,res_hours_9,...,res_hours_63,res_hours_64,res_hours_65,res_hours_66,res_hours_67,res_hours_68,res_hours_69,res_hours_70,res_hours_71,res_hours_72
3,141233,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,141276,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,141289,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,141436,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,141451,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857915,3353251,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
857949,3353251,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
857958,3353251,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
857960,3353251,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Apacheapsvar 表格处理

In [53]:
# 透析数据处理 apacheAspVar表格处理
apacheapsvar = pd.read_csv('./data/apacheApsVar.csv')
# Index(['apacheapsvarid', 'patientunitstayid', 'intubated', 'vent', 'dialysis',
#        'eyes', 'motor', 'verbal', 'meds', 'urine', 'wbc', 'temperature',
#        'respiratoryrate', 'sodium', 'heartrate', 'meanbp', 'ph', 'hematocrit',
#        'creatinine', 'albumin', 'pao2', 'pco2', 'bun', 'glucose', 'bilirubin',
#        'fio2'],
#       dtype='object')
apacheapsvar = apacheapsvar[['patientunitstayid', 'vent', 'intubated', 'dialysis']]
apacheapsvar

,patientunitstayid,vent,intubated,dialysis
0,141168,0,0,0
1,141178,0,0,0
2,141194,0,0,0
3,141197,0,0,0
4,141203,1,0,0
...,...,...,...,...
171172,3353235,0,0,0
171173,3353237,0,0,0
171174,3353251,1,1,0
171175,3353254,0,0,0
